In [1]:
# coding: utf-8
import numpy as np
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.autograd import Variable
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models

In [2]:
# Setup
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)

cpu


In [4]:
# Preprocess
pretrain_image_size = 224

transform_train = transforms.Compose(
    [transforms.RandomRotation(30),
     transforms.RandomHorizontalFlip(),
     torchvision.transforms.Resize(pretrain_image_size),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
transform_test = transforms.Compose(
    [torchvision.transforms.Resize(pretrain_image_size),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

In [5]:
# Data
trainset = torchvision.datasets.CIFAR100(root='./data', train=True,
                                        download=True, 
transform=transform_train)
trainloader = DataLoader(trainset, batch_size=100, shuffle=True, 
num_workers=2)

testset = torchvision.datasets.CIFAR100(root='./data', train=False,
                                       download=True, 
transform=transform_test)
testloader = DataLoader(testset, batch_size=100, shuffle=False, 
num_workers=2)

Files already downloaded and verified
Files already downloaded and verified


In [3]:
net = models.resnet18(pretrained=True)
net.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001)

Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/Clue/.torch/models/resnet18-5c106cde.pth
100.0%


In [6]:
# Train

net.train()
for epoch in range(10):  # loop over the dataset multiple times

    net.train()
    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data
        inputs, labels = inputs.to(device), labels.to(device)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()

        for group in optimizer.param_groups:
            for p in group['params']:
                state = optimizer.state[p]
                if('step' in state and state['step']>=1024):
                    state['step'] = 1000

        optimizer.step()

        #print statistics
#         running_loss += loss.item()
#         if i % 10 == 9:    # print every 10 mini-batches
#             print('[%d, %5d] loss: %.3f' %
#                  (epoch + 1, i + 1, running_loss / 10))
#             running_loss = 0.0

    net.eval()
    correct = 0
    total = 0
    with torch.no_grad():
        for data in testloader:
            images, labels = data
            images, labels = images.to(device), labels.to(device)

            outputs = net(images)
            _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (predicted == labels).sum().item()

    print('Accuracy of the network on the 10000 test images: %f %%' % (
        100 * correct / total))


print('Finished Training')

[1,    10] loss: 9.558
[1,    20] loss: 9.388
[1,    30] loss: 9.042


Process Process-2:
Process Process-1:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/Users/Clue/anaconda/envs/Design/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/Clue/anaconda/envs/Design/lib/python3.5/multiprocessing/process.py", line 252, in _bootstrap
    self.run()
  File "/Users/Clue/anaconda/envs/Design/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Clue/anaconda/envs/Design/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/Users/Clue/anaconda/envs/Design/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAGER_STATUS_CHECK_INTERVAL)
  File "/Users/Clue/anaconda/envs/Design/lib/python3.5/site-packages/torch/utils/data/dataloader.py", line 96, in _worker_loop
    r = index_queue.get(timeout=MANAG

KeyboardInterrupt: 

In [ ]:
# # Test

# net.eval()
# correct = 0
# total = 0
# with torch.no_grad():
#     for data in testloader:
#         images, labels = data
#         images, labels = images.to(device), labels.to(device)

#         outputs = net(images)
#         _, predicted = torch.max(outputs.data, 1)
#         total += labels.size(0)
#         correct += (predicted == labels).sum().item()

# print('Test Set Accuracy: %f %%' % (
#     100 * correct / total))